In [1]:
import numpy as np
from dataset import BooleanDataset

In [41]:
class PTA():
    def __init__(self, epochs, task = "OR"):
        self.task = task
        self.seen_params = []
        self.epochs = epochs

    def _init_params(self):
        self.bias = 1
        self.w0 = self.bias
        self.w1 = 0
        if self.task != "NOT":
            self.w2 = 0

    def _update_params(self, X, y, y_preds):
        for i in range(len(y)):
            if y[i] != y_preds[i]:
                error = y[i] - y_preds[i]
                self.w0 += error * X[i][0]
                self.w1 += error * X[i][1]
                if self.task != "NOT":
                    self.w2 += error * X[i][2]

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        n = X.shape[0]
        self._init_params()
        converged = False
        i = 1
        while i<self.epochs and not converged:
            y_preds = self.predict(X)
            converged = (n == (y_preds==y).sum())
            self._update_params(X, y, y_preds)
            w0, w1 = self.w0, self.w1

            if self.task != "NOT":
                w2 = self.w2
                print("Iteration ", i, " -> w0: ", w0, "| w1: ", w1, "| w2: ", w2, " | Accuracy: ", 100*(y_preds==y).sum()/n)
            else:
                print("Iteration ", i, " -> w0: ", w0, "| w1: ", w1, " | Accuracy: ", 100*(y_preds==y).sum()/n)

            if converged:
                print("Convergence reached. Hyperplane distributing the two classes has been found.")
                break
            if self.task != "NOT":
                self.seen_params.append([w0, w1, w2])
            else:
                self.seen_params.append([w0, w1])
            i+=1
        if not converged:
            print(f"Convergence not reached after {self.epochs} epochs. Consider using a larger number of epochs or a linearly separable dataset.")

    def predict(self, X):
        if self.task != "NOT":
            w = np.array([self.w0, self.w1, self.w2])
        else:
            w = np.array([self.w0, self.w1])
        y_preds = []
        for each in X:
            activation = np.sum(np.matmul(w.T, each))
            if activation>=0:
                y_preds.append(1)
            else:
                y_preds.append(-1)
        y_preds = np.array(y_preds)

        return y_preds
    
    def get_params(self):
        return self.seen_params[-1]

In [42]:
def save_model(model, filename):
    import pickle
    with open(f"Model_Saves/{filename}", 'wb') as f:
        pickle.dump(model, f)

def load_model(filename):
    import pickle
    with open(f"Model_Saves/{filename}", 'rb') as f:
        return pickle.load(f)

### AND

In [43]:
and_dataset = BooleanDataset("AND")
and_dataset.generate_data()

In [44]:
X, y = and_dataset.data, and_dataset.labels

In [45]:
X, y

([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]], [-1, -1, -1, 1])

In [46]:
and_pta = PTA(epochs=10)

In [47]:
and_pta.fit(X, y)

Iteration  1  -> w0:  -5 | w1:  -2 | w2:  -2  | Accuracy:  25.0
Iteration  2  -> w0:  -3 | w1:  0 | w2:  0  | Accuracy:  75.0
Iteration  3  -> w0:  -1 | w1:  2 | w2:  2  | Accuracy:  75.0
Iteration  4  -> w0:  -5 | w1:  0 | w2:  0  | Accuracy:  50.0
Iteration  5  -> w0:  -3 | w1:  2 | w2:  2  | Accuracy:  75.0
Iteration  6  -> w0:  -3 | w1:  2 | w2:  2  | Accuracy:  100.0
Convergence reached. Hyperplane distributing the two classes has been found.


In [49]:
save_model(and_pta, "and_pta.pickle")

### OR

In [50]:
or_dataset = BooleanDataset("OR")
or_dataset.generate_data()

In [51]:
X, y = or_dataset.data, or_dataset.labels

In [52]:
X, y

([[1, 0, 0], [1, 0, 1], [1, 1, 0], [1, 1, 1]], [-1, 1, 1, 1])

In [53]:
or_pta = PTA(epochs=10)

In [54]:
or_pta.fit(X, y)

Iteration  1  -> w0:  -1 | w1:  0 | w2:  0  | Accuracy:  75.0
Iteration  2  -> w0:  5 | w1:  4 | w2:  4  | Accuracy:  25.0
Iteration  3  -> w0:  3 | w1:  4 | w2:  4  | Accuracy:  75.0
Iteration  4  -> w0:  1 | w1:  4 | w2:  4  | Accuracy:  75.0
Iteration  5  -> w0:  -1 | w1:  4 | w2:  4  | Accuracy:  75.0
Iteration  6  -> w0:  -1 | w1:  4 | w2:  4  | Accuracy:  100.0
Convergence reached. Hyperplane distributing the two classes has been found.


In [56]:
save_model(or_pta, "or_pta.pickle")

### NOT

In [57]:
not_dataset = BooleanDataset("NOT")
not_dataset.generate_data()

In [58]:
X, y = not_dataset.data, not_dataset.labels

In [59]:
X, y

([[1, 0], [1, 1]], [-1, -1])

In [60]:
not_pta = PTA(epochs=10, task="NOT")

In [61]:
not_pta.fit(X, y)

Iteration  1  -> w0:  -3 | w1:  -2  | Accuracy:  0.0
Iteration  2  -> w0:  -3 | w1:  -2  | Accuracy:  100.0
Convergence reached. Hyperplane distributing the two classes has been found.


In [62]:
save_model(not_pta, "not_pta.pickle")